In [14]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyecharts.render.engine import render_notebook
from pyecharts.charts import Bar
from pyecharts.charts import Scatter
from pyecharts import options as opts
from itertools import zip_longest

In [15]:
#获得boss直聘公司信息
List1 = list()
List2 = list()
List3 = list()
for i in range(1, 9):
    # 创建浏览器驱动
    service = webdriver.EdgeService(
        executable_path='C:/Program Files (x86)/Microsoft/Edge/Application/msedgedriver.exe')
    driver = webdriver.Edge(service=service)
    # 打开网页
    driver.get("https://www.zhipin.com/gongsi/_zzz_c101010100/?page={}&ka=page-{}".format(i, i))

    # 等待动态内容加载完成
    wait = WebDriverWait(driver, 10)
    dynamic_content = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='wrap']")))

    # 获取整个 HTML 文档
    html_content = driver.page_source

    # 获取 ::before 后面的内容
    before_content = html_content.split("::before")[-1]


    #beautifulsoup解析
    Soup = BeautifulSoup(before_content)
    scop = Soup.find('div', class_="company-tab-box company-list")
    GONGSI = scop.find_all('h4')
    zhiwei = scop.find_all('p')
    shuxing=scop.find_all('span')
    print(GONGSI)
    print('-' * 60)
    print(zhiwei)
    List1.append(GONGSI)
    List2.append(zhiwei)
    List3.append(shuxing)
# 关闭浏览器
driver.quit()



[<h4>百度</h4>, <h4>快手</h4>, <h4>华为</h4>, <h4>BOSS直聘</h4>, <h4>阿里巴巴集团</h4>, <h4>好未来</h4>, <h4>滴滴出行</h4>, <h4>网易</h4>, <h4>中软国际</h4>, <h4>汽车之家</h4>, <h4>高德地图</h4>, <h4>腾讯</h4>, <h4>英雄互娱</h4>, <h4>完美世界</h4>, <h4>易车</h4>, <h4>中科软</h4>, <h4>四维图新</h4>, <h4>神州租车</h4>, <h4>金山云</h4>, <h4>奇虎360</h4>, <h4>纷享销客</h4>, <h4>饿了么</h4>, <h4>永航科技</h4>, <h4>亚信科技</h4>, <h4>货拉拉科技</h4>, <h4>美团网</h4>, <h4>Keep</h4>, <h4>掌阅科技</h4>]
------------------------------------------------------------
[<p>热招 <span class="vline"></span> <u class="h">高级算法工程师</u> 20-40K·16薪</p>, <p>热招 <span class="vline"></span> <u class="h">数据开发工程师</u> 30-50K·16薪</p>, <p>热招 <span class="vline"></span> <u class="h">硬件工程师（北京OD）</u> 15-30K·14薪</p>, <p>热招 <span class="vline"></span> <u class="h">用户运营</u> 3-4K</p>, <p>热招 <span class="vline"></span> <u class="h">JAVA工程师</u> 20-27K</p>, <p>热招 <span class="vline"></span> <u class="h">UI设计实习生</u> 200-240元/天</p>, <p>热招 <span class="vline"></span> <u class="h">算法工程师</u> 25-50K·15薪</p>, <p>热招 <span cl

In [20]:
#爬取的结果是一个由beautifulsoup组成的列表，为了继续提取信息转为txt并保存
# 读取爬取的列表内容并移入output.txt
with open('output.txt', 'w', encoding='utf-8') as file:
    for item in range(len(List2)):
        file.write(str(List2[item]))

#初步获取职位信息(List1)
with open('output.txt','r',encoding='utf-8') as file:
    file1=BeautifulSoup(file,'html.parser')
    zhiwei=file1.find_all('u',class_="h")

with open('output.txt','r',encoding='utf-8') as file:
    file1=BeautifulSoup(file,'html.parser')
    zhiwei=file1.find_all('u',class_="h")


#初步获取薪资信息(List1)

import re
    #定义re的匹配模式
pattern = r'(\d+-\d+K)'
    #定义一个获取薪资的函数
def get_money(filename):
    #初始化
    salaries=[]
    with open(filename, 'r',encoding='utf-8') as file:
    #逐行读取文本
        for line in file:
            #re匹配
            matches = re.findall(pattern, line)
            #加入列表
            salaries.extend(matches)
        return salaries
filename = "output.txt"
moneys = get_money(filename)


    #初始化
position2 = []
Salary2 = []
result2 = []
kind2=[]
# 对职位信息进一步提取
for zhi in range(len(zhiwei)):
    position1 = zhiwei[zhi].text
    position2.append(position1)
# 对薪资进一步提取
for money in range(len(moneys)):
    Salary1 = moneys[money]
    Salary2.append(Salary1)
#对公司进一步提取（List2）
for i in range(len(List1)):
    a = str(List1[i])
    pattern = "<h4>(.*?)</h4>"
    result1 = re.findall(pattern, a)
    result2.append(result1)

for i in range(len(List3)):
    b = str(List3[i])
    patterns = "<span>(.*?)</span>"
    kind1 = re.findall(patterns, b)
    kind2.append(kind1)


for sublist in kind2:  
    if '...' in sublist:  
        sublist.remove('...')  

all_rows = []  # 创建一个空列表来收集所有的字典  
  
for sublist in kind2:  
    # 使用列表推导式来创建包含每对公司属性的字典列表，同时确保不会超出索引范围  
    rows = [{ '属性1':sublist[i], '属性2':sublist[i+1]} for i in range(0, len(sublist) - 1, 2) if i + 1 < len(sublist)]  
      
    # 将这个子列表的字典列表添加到总列表中  
    all_rows.extend(rows)  
  
# 使用收集到的所有字典列表创建最终的DataFrame  
kind = pd.DataFrame(all_rows) 


#创建公司列
gongsi=pd.DataFrame(result2)
conpany = gongsi.stack()
conpany = conpany.reset_index(drop=True)
conpany=pd.Series(conpany,name='conpany')
#创建职位列
position=pd.Series(position2,name='position')
#创建薪资列
Salary=pd.Series(Salary2,name='Salary')
#组合成表格存入CSV
df = pd.concat([conpany,position,Salary],axis=1)#.to_csv('boss直聘.csv')


# 处理空值
Salary = Salary.fillna('0 0')

#创建函数用于读取数字
def get_averageSalary(salary):
    # 匹配数字
    numbers = re.findall(r'\d+', salary)
    #如果没有两个数字就返回None
    if len(numbers) != 2:
        return None
    # 将字符串转换为整数
    num1, num2 = int(numbers[0]), int(numbers[1])
    # 计算均值
    return (num1 + num2) / 2


mid_Salary = []
#遍历薪资列取出数字并且计算均值
for q in range(0, len(Salary)):

    avrage_salary = get_averageSalary(Salary[q])
    if avrage_salary is not None:  # 确保计算得到的平均薪资不是 None
        mid_Salary.append(avrage_salary)
mid_Salary = pd.Series(mid_Salary, name='avg_salary')
#整合进最后的Dataframe
boss = pd.concat([conpany,position,Salary,mid_Salary],axis=1)#.to_csv('北京boss直聘.csv')

max_len = max(len(boss), len(kind))  
boss = pd.concat([boss, pd.DataFrame(index=range(max_len - len(boss)))]).reset_index(drop=True)  
kind = pd.concat([kind, pd.DataFrame(index=range(max_len - len(kind)))]).reset_index(drop=True)  
  
# 现在 df_boss 和 df_kind 有相同的行数，可以直接使用 concat 结合  
df_combined = pd.concat([boss, kind], axis=1)

In [22]:
kind2

[['已上市',
  '互联网',
  '已上市',
  '社交网络与媒体',
  '不需要融资',
  '计算机软件',
  '已上市',
  '人力资源服务',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  'D轮及以上',
  '互联网',
  '已上市',
  '互联网',
  '已上市',
  '计算机软件',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  '不需要融资',
  '互联网',
  '已上市',
  '游戏',
  '已上市',
  '游戏',
  '已上市',
  '互联网',
  '已上市',
  '计算机软件',
  '已上市',
  '互联网',
  '已上市',
  '生活服务（O2O）',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  'D轮及以上',
  '互联网',
  '不需要融资',
  '生活服务（O2O）',
  '不需要融资',
  '游戏',
  '已上市',
  '计算机软件',
  'D轮及以上',
  '互联网',
  '已上市',
  '生活服务（O2O）',
  '已上市',
  '互联网',
  '已上市',
  '互联网'],
 ['D轮及以上',
  '计算机服务',
  '已上市',
  '电子商务',
  'D轮及以上',
  '移动互联网',
  '已上市',
  '计算机软件',
  '已上市',
  '计算机软件',
  '已上市',
  '移动互联网',
  'C轮',
  '电子商务',
  'D轮及以上',
  '互联网',
  'C轮',
  '互联网',
  '已上市',
  '互联网',
  'D轮及以上',
  '互联网',
  'D轮及以上',
  '互联网',
  '不需要融资',
  '互联网',
  'C轮',
  '信息安全',
  '已上市',
  '互联网金融',
  'C轮',
  '互联网',
  '已上市',
  '培训/辅导机构',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  '已上市',
  '计算机软件',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  '已上市',
  '互联网',
  '不需要融资

In [18]:
data

NameError: name 'data' is not defined

In [ ]:
data = data.drop(columns=data.columns[0])  

In [ ]:
print(data.columns)

In [ ]:
data= data.dropna()  

In [ ]:
def extract_salary_range(salary_str):  
    min_max = salary_str.split('-')  
    min_salary = int(min_max[0].replace('K', '') or min_max[0]) * (1000 if 'K' in min_max[0] else 1)  
    max_salary = int(min_max[1].replace('K', '') or min_max[1]) * (1000 if 'K' in min_max[1] else 1)  
    return pd.Series({'最小值': min_salary, '最大值': max_salary})  

data = data.join(data？['Salary'].apply(extract_salary_range)) 

In [ ]:
data['最大值'] = data['最大值'] / 1000 

In [ ]:
data['最大值'] = data['最大值'].round().astype(int)

In [ ]:
data = data.drop(columns=['position', 'Salary'])

In [ ]:
cities = pd.Series(['------'] * len(data))  
data.insert(0, '城市', cities)  

In [ ]:
data